# Great Expectations para Calidad de Datos

Este notebook demuestra cómo usar Great Expectations para validar datos.

**Referencia:** [Great Expectations](../calidad/herramientas/great-expectations-para-calidad.md)

**Objetivos:**
- Crear expectativas sobre datos
- Validar automáticamente
- Generar reportes de calidad

## 1. Instalación

In [ ]:
# Descomenta para instalar
# !pip install great-expectations

print("💡 Para instalar: pip install great-expectations")

## 2. Importar librerías

In [ ]:
import pandas as pd
import numpy as np

# Great Expectations
try:
    import great_expectations as gx
    print("✅ Great Expectations importado")
except ImportError:
    print("⚠️ Great Expectations no instalado. Ejecuta: pip install great-expectations")

## 3. Crear datos de ejemplo

In [ ]:
# Crear datos de ejemplo
np.random.seed(42)
df = pd.DataFrame({
    'id': range(1, 101),
    'nombre': [f'Usuario {i}' for i in range(1, 101)],
    'edad': np.random.randint(18, 80, 100),
    'email': [f'user{i}@email.com' for i in range(1, 101)],
    'salario': np.random.uniform(30000, 100000, 100)
})

print(f"✅ Datos creados: {len(df)} registros")
df.head()

## 4. Usar Great Expectations (si está instalado)

In [ ]:
try:
    # Crear contexto (para notebooks, usa contexto estándar)
    context = gx.get_context()
    
    # Crear datasource
    datasource = context.sources.add_pandas("my_datasource")
    
    # Crear asset
    asset = datasource.add_dataframe_asset(name="my_dataframe")
    
    # Crear validator
    validator = context.get_validator(
        datasource_name="my_datasource",
        data_asset_name="my_dataframe",
        dataframe=df
    )
    
    print("✅ Validator creado")
    
    # Definir expectativas
    validator.expect_column_values_to_not_be_null("nombre")
    validator.expect_column_values_to_be_between("edad", min_value=0, max_value=120)
    validator.expect_column_values_to_be_unique("id")
    validator.expect_column_values_to_be_between("salario", min_value=0)
    
    print("✅ Expectativas definidas")
    
    # Validar
    results = validator.validate()
    print("\n✅ Validación completada")
    print(f"\nResultados: {results.success}")
    
    # Mostrar detalles de las expectativas
    if hasattr(results, 'results'):
        print(f"\n📊 Expectativas ejecutadas: {len(results.results)}")
        for i, result in enumerate(results.results, 1):
            estado = "✅" if result.success else "❌"
            print(f"   {estado} {result.expectation_config.expectation_type}")
    
except (NameError, AttributeError) as e:
    print(f"\n💡 Great Expectations no está instalado o hay un error de configuración.")
    print(f"   Error: {str(e)}")
    print("\nEjemplo de código que usarías:")
    print("""
    import great_expectations as gx
    
    context = gx.get_context()
    datasource = context.sources.add_pandas("my_datasource")
    asset = datasource.add_dataframe_asset(name="my_dataframe")
    
    validator = context.get_validator(
        datasource_name="my_datasource",
        data_asset_name="my_dataframe",
        dataframe=df
    )
    
    # Expectativas
    validator.expect_column_values_to_not_be_null('nombre')
    validator.expect_column_values_to_be_between('edad', 0, 120)
    
    # Validar
    results = validator.validate()
    """)

## 5. Validación manual (alternativa sin GX)

In [ ]:
# Validación manual equivalente
def validar_datos(df):
    """Valida datos manualmente."""
    resultados = {}
    
    # Expectativa 1: nombre no nulo
    resultados['nombre_no_nulo'] = df['nombre'].notna().all()
    
    # Expectativa 2: edad en rango
    resultados['edad_en_rango'] = df['edad'].between(0, 120).all()
    
    # Expectativa 3: id único
    resultados['id_unico'] = df['id'].is_unique
    
    # Expectativa 4: salario positivo
    resultados['salario_positivo'] = (df['salario'] > 0).all()
    
    return resultados

resultados = validar_datos(df)
print("=== RESULTADOS DE VALIDACIÓN ===")
for expectativa, resultado in resultados.items():
    estado = "✅" if resultado else "❌"
    print(f"{estado} {expectativa}: {resultado}")

print(f"\n✅ Todas las validaciones pasaron: {all(resultados.values())}")